<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: KNN #


##Inicialización##

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

In [3]:
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

Cloning into 'fiumark'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 104 (delta 54), reused 44 (delta 17), pack-reused 0
Receiving objects: 100% (104/104), 27.31 KiB | 527.00 KiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


##Pre procesamiento##

Para KNN es importante escalar los datos. 

In [4]:
pd.options.display.max_columns = None

df = pp.get_dataset()
df = pp.knn_preprocessing(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)


##Entrenamiento##

In [5]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_neighbors': np.arange(1, 100),
    'weights': ['uniform', 'distance'],
    'metric': ['cosine','minkowski'],
    'algorithm':['ball_tree','kd_tree','brute','auto'],
    'leaf_size': [30,40,50]
}

clf = KNeighborsClassifier()

rgscv = RandomizedSearchCV(
    clf, params, n_iter=200, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [6]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.841589880414191
Best params {'weights': 'distance', 'n_neighbors': 27, 'metric': 'minkowski', 'leaf_size': 30, 'algorithm': 'kd_tree'}


##Evaluación del modelo##

In [7]:
y_pred = rgscv.predict(X_test)

###Precision recall y F1-score###

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.81      0.92      0.86        95
         1.0       0.85      0.68      0.76        66

    accuracy                           0.82       161
   macro avg       0.83      0.80      0.81       161
weighted avg       0.82      0.82      0.82       161



###Accuracy###

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8198757763975155

###Auc-Roc###

In [10]:
from sklearn.metrics import roc_auc_score
auc_roc = roc_auc_score(y_test, y_pred)
auc_roc

0.7988038277511961

##Predicción##

In [11]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = pp.knn_preprocessing(X_holdout)
prediction = pd.DataFrame(rgscv.predict(X),columns=['volveria'],dtype=int)
pred_formatted = pd.concat([X_holdout.id_usuario,prediction],axis=1)
pred_formatted.to_csv('knn.csv')
pred_formatted

,id_usuario,volveria
0,650,0
1,234,0
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,0
